# Classify Category (Decision Tree)
##### Mr.Jakkrit Sonsanit

***

### Import Library 

In [1]:
import pandas as pd
import numpy as np
from pythainlp.corpus.common import thai_words
from pythainlp.tokenize import dict_trie, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

### Import data

In [3]:
df = pd.read_csv("Expenses.csv")
df.tail()

,date,text,money,cate
57,10/10/2019,โอ้เด้ง,43.0,Food
58,10/10/2019,ป็อปคอน,20.0,Food
59,11/10/2019,ค่าเหล้า,120.0,Entertainment
60,11/10/2019,ค่ากระเพรา,50.0,Food
61,11/10/2019,ถั่ว,10.0,Food


### Create Corpus

In [4]:
custom_dict = set(thai_words())
custom_dict.add('ราเมง')
custom_dict.add('คาปูชิโน่')
trie = dict_trie(dict_source=custom_dict)

corpus = []
for i in df.text:
    for j in word_tokenize(i, engine='dict', custom_dict=trie):
        if j not in corpus:
            corpus.append(j)
corpus
print(corpus)

['นม', 'ช็อคโกแลต', 'ข้าว', 'เที่ยง', 'น้ำ', 'กระเจี๊ยบ', 'ชา', 'ข้าวเย็น', 'ค่า', 'อินเตอร์เน็ต', 'ค่าโทรศัพท์', 'หมา', 'ล่า', 'ไอติม', 'ตัดผม', 'ซาลาเปา', 'ไส้กรอก', 'ดอย', 'คำ', 'ใบมีดโกน', 'ห้อง', 'Netflix', 'เงินเก็บ', 'ราเมง', 'นมเปรี้ยว', 'ขนมปัง', 'ขนม', 'arl', 'mrt', 'ซื้อ', 'กางเกง', 'ถั่ว', 'มาม่า', 'อิชิตัน', 'คาปูชิโน่', 'viu', 'บัวลอย', 'ค่ารถ', 'โกโก้', 'หนัง', 'เติม', 'เกม', 'โอ้', 'เด้ง', 'ป็อป', 'คอน', 'เหล้า', 'กระ', 'เพรา']


### Create Bag of Word

In [5]:
BOW = [list() for i in range(len(df.text))]
l = 0
count = 1
for i in df.text:
    tmp = word_tokenize(i, engine='dict', custom_dict=trie)
    for j in corpus:

        if j in tmp:

            BOW[l].append(1)
            tmp.remove(j)
            
        else:
            BOW[l].append(0)
        
    if len(tmp) != 0:
        BOW[l].append(len(tmp))
    elif len(tmp) == 0:
        BOW[l].append(0)
    l += 1

### Train Decision Tree model

In [6]:
ytarget = df.cate
xtrain = BOW

dtree = DecisionTreeClassifier()
dtree.fit(X=xtrain,y=ytarget)
predictiontree = dtree.predict(xtrain)

### Create Bag of Word form Test data

In [16]:
txt = ['ค่าหนัง', 'นม', 'ขนม']
BOW_t = [list() for i in range(len(txt))]
l = 0
for i in txt:
    tmp = word_tokenize(i, engine='dict', custom_dict=trie)
    for j in corpus:

        if j in tmp:

            BOW_t[l].append(1)
            tmp.remove(j)
        else:
            BOW_t[l].append(0)
        
    if len(tmp) != 0:
        BOW_t[l].append(len(tmp))
    elif len(tmp) == 0:
        BOW_t[l].append(0)
    l += 1

### Predict Category 

In [17]:
predictiontree = dtree.predict(BOW_t)
predictiontree

array(['Entertainment', 'Food', 'Food'], dtype=object)